# CEK 기계에 기본제공연산 추가
CEK 기계는 80년대에 Friedman과 Felleisen이 SECD 기계(Landin 1964)를 더 말끔하게 정리한 것이다.

여기서는 지난번 노트북에서 살펴본 CEK 기계에 기본제공연산(pritivmve operation)을 추가해 보자.
강의 일정상 지나친 복잡함은 피하기 위해 정수값들에 대한 기본연산만을 고려하기로 하자.

###### 참고자료
* https://www.cs.bham.ac.uk/~hxt/2015/compilers/compiling-functional.pdf
* https://en.wikipedia.org/wiki/SECD_machine
* https://www.brics.dk/RS/03/33/
* https://pages.cpsc.ucalgary.ca/~robin/FMCS/FMCS2014/Prashant_talk.pdf
* https://github.com/kseo/secd

In [16]:
:opt no-lint

In [17]:
type Nm = String

data Exp = Var Nm         -- x
         | App Exp Exp    -- e1 e2
         | Lam Nm Exp     -- \x.e
         | Lit Int        -- n
         | Prm Prim [Exp] -- f(x1,...,xn)
         deriving Show

data Prim = Suc -- +1을 하는 인자 1개짜리 기초연산
          | Add -- +를 하는 인자 2개짜리 기초연산
          deriving Show

In [18]:
type CEK = (Code,Env,Kont)

data Code = OpE Exp     -- 계산이 끝나지 않은 식
          | OpV Value   -- 계산이 끝난 값
          | OpArg       -- primitive 연산에 대한 인자를 처리
          | OpCall      -- 인자가 다 채워진 상태에서 primtive 연산 발동
          deriving Show
type Env  = [(Nm,Value)]
type Kont = [Frame]

data Value = Vint Int      -- n
           | Clos Exp Env  -- < \x.e, env >
           deriving Show

data Frame = FrV Value   -- (v O)
           | FrE Exp Env -- (O e env)
           | FrP Prim [Value] [Exp] Env -- (f [vi,..] O [ei,...] env)
           deriving Show

$ \big\langle x ~\big|~ \sigma ~\big|~ \kappa \big\rangle
  \longrightarrow
  \big\langle \sigma(x) ~\big|~ \sigma ~\big|~ \kappa \big\rangle $

$ \big\langle e_1~e_2 ~\big|~ \sigma ~\big|~ \kappa \big\rangle
  \longrightarrow
  \big\langle e_1 ~\big|~ \sigma ~\big|~  (\bigcirc\,e_2\;\sigma),\kappa \big\rangle $

$ \big\langle \lambda x.e ~\big|~ \sigma ~\big|~ \kappa \big\rangle
  \longrightarrow 
  \big\langle \langle\lambda x.e,\sigma\rangle ~\big|~ \sigma ~\big|~ \kappa \big\rangle $

$ \big\langle v_1 ~\big|~ \sigma_1 ~\big|~ (\bigcirc\,e_2\;\sigma_2),\kappa  \big\rangle
  \longrightarrow
  \big\langle e_2 ~\big|~ \sigma_2 ~\big|~ (v_1\,\bigcirc),\kappa  \big\rangle $

$ \big\langle v_2 ~\big|~ \sigma_2 ~\big|~ (\langle\lambda x.e,\sigma_1\rangle\,\bigcirc),\kappa  \big\rangle
  \longrightarrow
  \big\langle e ~\big|~ [x\mapsto v_2]\sigma_1 ~\big|~ \kappa \big\rangle $

$ \big\langle f(\bar{e}) ~\big|~ \sigma ~\big|~ \kappa \big\rangle
  \longrightarrow 
  \big\langle OpArg ~\big|~ \sigma ~\big|~  (f~\bar{e}\;\sigma),\kappa \big\rangle
  $

$ \big\langle OpArg ~\big|~ \sigma' ~\big|~  (f~\bar{v}~e~\bar{e}\;\sigma),\kappa \big\rangle
\longrightarrow
\big\langle e ~\big|~ \sigma ~\big|~  (f~\bar{v}\bigcirc\bar{e}\;\sigma),\kappa \big\rangle $

$\big\langle v ~\big|~ \sigma' ~\big|~  (f~\bar{v}\bigcirc \bar{e}\;\sigma),\kappa \big\rangle 
\longrightarrow
\big\langle OpArg ~\big|~ \sigma' ~\big|~  (f~\bar{v}~v~\bar{e}\;\sigma),\kappa \big\rangle
$

$\big\langle OpArg ~\big|~ \sigma' ~\big|~  (f~\bar{v}\;\sigma),\kappa \big\rangle 
\longrightarrow
\big\langle OpCall ~\big|~ \sigma' ~\big|~  (f~\bar{v}\;\sigma),\kappa \big\rangle
$

$\big\langle OpCall ~\big|~ \sigma' ~\big|~  (f~\bar{v}\;\sigma),\kappa \big\rangle
\longrightarrow
\big\langle \texttt{callPrim}f(\bar{v}) ~\big|~ \sigma ~\big|~  \kappa \big\rangle
$

In [26]:
lookup' x env = case lookup x env of Just v  -> v
                                     Nothing -> error("x is unknown")

-- 기본제공연산이 호출되었을 때 동작 정의
callPrim Suc [Vint n] = Vint (n+1)
callPrim Add [Vint n2, Vint n1] = Vint (n1+n2)

step :: CEK -> Maybe CEK
step (OpE(Var x),     env, k) = Just (OpV(lookup' x env), env, k)
step (OpE(App e1 e2), env, k) = Just (OpE e1, env, FrE e2 env : k)
step (OpE(e@Lam{}),   env, k) = Just (OpV(Clos e env), env, k)
step (OpE(Lit n),     env, k) = Just (OpV(Vint n), env, k)
step (OpE(Prm f es),  env, k) = Just (OpArg, env, FrP f [] es env : k) -- 기본제공연산식 처리 시작
step (OpV v,  env1, FrE e2 env2 : k) = Just (OpE e2, env2, FrV v : k)
step (OpV v,  env1, FrV(Clos (Lam x e) env2) : k) = Just (OpE e, (x,v) : env2, k)
step (OpArg,  env1, FrP f vs (e:es) env : k) = Just (OpE e, env, FrP f vs es env : k) -- 다음 인자값 계산
step (OpV v,  env1, FrP f vs es env : k) =  Just (OpArg, env1, FrP f (vs++[v]) es env : k) -- 계산된 인자값 프레임에 추가
step (OpArg,  env1, FrP f vs [] env : k) = Just (OpCall, env1, FrP f vs [] env : k) -- 다음 인자값 계산 시도하지만 더 이상 인자 없는 경우 처리
step (OpCall, env1, FrP f vs [] env : k) = Just (OpV(callPrim f vs), env, k) -- 모든 인자 계산 후 기본제공연산 호출
step _ = Nothing

In [27]:
{-# LANGUAGE FlexibleInstances #-}
import IHaskell.Display
import Data.List (intersperse)

class TeX a where
  toTeX :: a -> String

instance TeX Exp where
  toTeX (Var x) = x
  toTeX (App e1 e2) = "("++toTeX e1++"\\;"++toTeX e2++")" 
  toTeX (Lam x e) = "(\\lambda{}"++x++"."++toTeX e++")"
  toTeX (Lit n) = show n
  toTeX (Prm f es) = "\\textsf{"++show f++"}"++toTeX es

instance TeX Code where
  toTeX (OpE e) = toTeX e
  toTeX (OpV v) = toTeX v
  toTeX (OpArg) = "\\texttt{arg}"
  toTeX (OpCall) = "\\texttt{call}"

instance TeX Value where
  toTeX (Vint n) = show n
  toTeX (Clos e env) = "\\langle{}" ++ toTeX e ++ "," ++ revTeX env ++ "\\rangle{}"

instance TeX Frame where
  toTeX (FrV v) = "("++toTeX v++"\\,\\bigcirc{})"
  toTeX (FrE e env) = "(\\bigcirc{}\\,"++toTeX e++"\\;"++revTeX env++")"
  toTeX (FrP f vs es env) = "(\\textsf{"++show f++"}"++revTeX vs++"\\,\\bigcirc{}\\,"++toTeX es++"\\;"++revTeX env++")"

instance TeX (Nm,Value) where
  toTeX (x,v) = x++"\\mapsto{}"++toTeX v

instance TeX CEK where
  toTeX (c,env,k) = "\\big\\langle{}"
                 ++ toTeX c ++ "\\;\\big|\\;" ++ revTeX env ++ "\\;\\big|\\;" ++ toTeX k
                 ++ "\\big\\rangle{}"

instance TeX a => TeX (Maybe a) where
  toTeX (Just x)= "\\texttt{Just}("++toTeX x++")"
  toTeX Nothing = "\\texttt{Nothing}"

instance TeX a => TeX [a] where
  toTeX xs = "[" ++ (concat . intersperse "," $ map toTeX xs) ++ "]"

revTeX = toTeX . reverse

newtype LaTeX a = LaTeX a

htmlTeX a = html $ "$"++toTeX a++"$"

instance TeX a => IHaskellDisplay (LaTeX a) where
  display (LaTeX a) = return $ Display [htmlTeX a]

이전 예제는 그대로 잘 동작한다.

In [28]:
fst_1_2 = App (App (Lam "x" $ Lam "y" $ Var "x") (Lit 1)) (Lit 2)
fst_1_2
LaTeX fst_1_2
LaTeX (Prm Suc[Lit 3])

App (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)) (Lit 2)

$(((\lambda{}x.(\lambda{}y.x))\;1)\;2)$

$\textsf{Suc}[3]$

In [29]:
import Data.Maybe

mapM_ print $ takeWhile isJust $ iterate (step =<<) (Just(OpE fst_1_2,[],[]))

Just (OpE (App (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)) (Lit 2)),[],[])
Just (OpE (App (Lam "x" (Lam "y" (Var "x"))) (Lit 1)),[],[FrE (Lit 2) []])
Just (OpE (Lam "x" (Lam "y" (Var "x"))),[],[FrE (Lit 1) [],FrE (Lit 2) []])
Just (OpV (Clos (Lam "x" (Lam "y" (Var "x"))) []),[],[FrE (Lit 1) [],FrE (Lit 2) []])
Just (OpE (Lit 1),[],[FrV (Clos (Lam "x" (Lam "y" (Var "x"))) []),FrE (Lit 2) []])
Just (OpV (Vint 1),[],[FrV (Clos (Lam "x" (Lam "y" (Var "x"))) []),FrE (Lit 2) []])
Just (OpE (Lam "y" (Var "x")),[("x",Vint 1)],[FrE (Lit 2) []])
Just (OpV (Clos (Lam "y" (Var "x")) [("x",Vint 1)]),[("x",Vint 1)],[FrE (Lit 2) []])
Just (OpE (Lit 2),[],[FrV (Clos (Lam "y" (Var "x")) [("x",Vint 1)])])
Just (OpV (Vint 2),[],[FrV (Clos (Lam "y" (Var "x")) [("x",Vint 1)])])
Just (OpE (Var "x"),[("y",Vint 2),("x",Vint 1)],[])
Just (OpV (Vint 1),[("y",Vint 2),("x",Vint 1)],[])

In [30]:
map LaTeX . takeWhile isJust $ iterate (step =<<) (Just(OpE fst_1_2,[],[]))

$\texttt{Just}(\big\langle{}(((\lambda{}x.(\lambda{}y.x))\;1)\;2)\;\big|\;[]\;\big|\;[]\big\rangle{})$

$\texttt{Just}(\big\langle{}((\lambda{}x.(\lambda{}y.x))\;1)\;\big|\;[]\;\big|\;[(\bigcirc{}\,2\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}(\lambda{}x.(\lambda{}y.x))\;\big|\;[]\;\big|\;[(\bigcirc{}\,1\;[]),(\bigcirc{}\,2\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{}\;\big|\;[]\;\big|\;[(\bigcirc{}\,1\;[]),(\bigcirc{}\,2\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}1\;\big|\;[]\;\big|\;[(\langle{}(\lambda{}x.(\lambda{}y.x)),[]\rangle{}\,\bigcirc{}),(\bigcirc{}\,2\;[])]\big\rangle{})$

...

$\texttt{Just}(\big\langle{}\langle{}(\lambda{}y.x),[x\mapsto{}1]\rangle{}\;\big|\;[x\mapsto{}1]\;\big|\;[(\bigcirc{}\,2\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}2\;\big|\;[]\;\big|\;[(\langle{}(\lambda{}y.x),[x\mapsto{}1]\rangle{}\,\bigcirc{})]\big\rangle{})$

$\texttt{Just}(\big\langle{}2\;\big|\;[]\;\big|\;[(\langle{}(\lambda{}y.x),[x\mapsto{}1]\rangle{}\,\bigcirc{})]\big\rangle{})$

$\texttt{Just}(\big\langle{}x\;\big|\;[x\mapsto{}1,y\mapsto{}2]\;\big|\;[]\big\rangle{})$

$\texttt{Just}(\big\langle{}1\;\big|\;[x\mapsto{}1,y\mapsto{}2]\;\big|\;[]\big\rangle{})$

In [31]:
Just(OpE(Prm Suc[Lit 3]),[],[])
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it

isJust (Just _) = True
isJust Nothing  = False

map LaTeX . takeWhile isJust . iterate (step =<<) $ Just(OpE(Prm Suc[Lit 3]),[],[])

Just (OpE (Prm Suc [Lit 3]),[],[])

Just (OpArg,[],[FrP Suc [] [Lit 3] []])

Just (OpE (Lit 3),[],[FrP Suc [] [] []])

Just (OpV (Vint 3),[],[FrP Suc [] [] []])

Just (OpArg,[],[FrP Suc [Vint 3] [] []])

...

$\texttt{Just}(\big\langle{}3\;\big|\;[]\;\big|\;[(\textsf{Suc}[]\,\bigcirc{}\,[]\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}3\;\big|\;[]\;\big|\;[(\textsf{Suc}[]\,\bigcirc{}\,[]\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}\texttt{arg}\;\big|\;[]\;\big|\;[(\textsf{Suc}[3]\,\bigcirc{}\,[]\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}\texttt{call}\;\big|\;[]\;\big|\;[(\textsf{Suc}[3]\,\bigcirc{}\,[]\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}4\;\big|\;[]\;\big|\;[]\big\rangle{})$

In [32]:
Just(OpE(Prm Add[Lit 3,Lit 4]),[],[])
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it
step =<< it

isJust (Just _) = True
isJust Nothing  = False

map LaTeX . takeWhile isJust . iterate (step =<<) $ Just(OpE(Prm Add[Lit 3,Lit 4]),[],[])

Just (OpE (Prm Add [Lit 3,Lit 4]),[],[])

Just (OpArg,[],[FrP Add [] [Lit 3,Lit 4] []])

Just (OpE (Lit 3),[],[FrP Add [] [Lit 4] []])

Just (OpV (Vint 3),[],[FrP Add [] [Lit 4] []])

Just (OpArg,[],[FrP Add [Vint 3] [Lit 4] []])

...

$\texttt{Just}(\big\langle{}4\;\big|\;[]\;\big|\;[(\textsf{Add}[3]\,\bigcirc{}\,[]\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}4\;\big|\;[]\;\big|\;[(\textsf{Add}[3]\,\bigcirc{}\,[]\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}\texttt{arg}\;\big|\;[]\;\big|\;[(\textsf{Add}[4,3]\,\bigcirc{}\,[]\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}\texttt{call}\;\big|\;[]\;\big|\;[(\textsf{Add}[4,3]\,\bigcirc{}\,[]\;[])]\big\rangle{})$

$\texttt{Just}(\big\langle{}7\;\big|\;[]\;\big|\;[]\big\rangle{})$